In [1]:
!pip install pyvis

In [2]:
import networkx as nx
import numpy as np
import pandas as pd
from pyvis.network import Network
import os
import requests
from requests.exceptions import SSLError
import zipfile

In [3]:
base_dir = '.'
workspace_dir = f'{base_dir}'
# os.makedirs(os.path.abspath(workspace_dir), exist_ok=True)

In [4]:
feed_url = 'https://api-public.odpt.org/api/v4/files/Toei/data/ToeiBus-GTFS.zip'

try:
  res = requests.get(feed_url, stream=True)
except SSLError:
  res = requests.get(feed_url, stream=True, verify=False)

with open(os.path.abspath(f'{workspace_dir}/gtfsjp.zip'), 'wb') as f:
  f.write(res.content)

  with zipfile.ZipFile(os.path.abspath(f'{workspace_dir}/gtfsjp.zip')) as zip:
    zip.extractall(os.path.abspath(f'{workspace_dir}/gtfsjp'))

    os.remove(os.path.abspath(f'{workspace_dir}/gtfsjp.zip'))

In [5]:
df_stops = pd.read_csv(f'{workspace_dir}/gtfsjp/stops.txt')
df_stops[['stop_id', 'stop_name']]

,stop_id,stop_name
0,0001-01,愛育クリニック前
1,0001-02,愛育クリニック前
2,0003-01,青戸車庫前
3,0003-02,青戸車庫前
4,0003-03,青戸車庫前
...,...,...
3712,2622-01,東京女子医大足立医療センター前
3713,2622-02,東京女子医大足立医療センター前
3714,2624-01,第二公園
3715,2625-04,菅原橋西


In [6]:
df_stop_times = pd.read_csv(f'{workspace_dir}/gtfsjp/stop_times.txt')
df_stop_times[['trip_id', 'stop_id', 'stop_sequence']]

,trip_id,stop_id,stop_sequence
0,00106-1-01-105-0913,0737-01,1
1,00106-1-01-105-0913,1239-01,2
2,00106-1-01-105-0913,2024-01,3
3,00106-1-01-105-0913,2579-01,4
4,00106-1-01-105-0913,0946-02,5
...,...,...,...
981655,74601-2-81-160-2000,0594-01,15
981656,74601-2-81-160-2000,2003-02,16
981657,74601-2-81-160-2000,2587-02,17
981658,74601-2-81-160-2000,2592-02,18


In [7]:
df_gtfs = pd.merge(df_stops, df_stop_times, on='stop_id')[['trip_id', 'stop_sequence', 'stop_id', 'stop_name']].sort_values(['trip_id', 'stop_sequence'], ignore_index=True)
df_gtfs

,trip_id,stop_sequence,stop_id,stop_name
0,00106-1-01-105-0913,1,0737-01,新橋駅前
1,00106-1-01-105-0913,2,1239-01,浜離宮前
2,00106-1-01-105-0913,3,2024-01,築地五丁目
3,00106-1-01-105-0913,4,2579-01,国立がん研究センター前
4,00106-1-01-105-0913,5,0946-02,築地六丁目
...,...,...,...,...
981655,74601-2-81-160-2000,15,0594-01,塩浜二丁目
981656,74601-2-81-160-2000,16,2003-02,塩浜東
981657,74601-2-81-160-2000,17,2587-02,深川八中南
981658,74601-2-81-160-2000,18,2592-02,東陽橋南


In [8]:
dict_gtfs = df_gtfs.T.to_dict()
dict_network = {}

index = -1

for i in dict_gtfs.values():
  index += 1

  if index == 0 or i['trip_id'] != df_gtfs['trip_id'][index - 1]:
    continue

  k = f"{df_gtfs['stop_id'][index - 1]}_{i['stop_id']}"

  if k in dict_network:
    dict_network[k]['weight'] += 1
    continue

  dict_network[k] = {
    'source_id': df_gtfs['stop_id'][index - 1],
    'target_id': i['stop_id'],
    'weight': 1,
    'source_name': df_gtfs['stop_name'][index - 1],
    'target_name': i['stop_name'],
  }

df_network = pd.DataFrame(dict_network.values(), index=dict_network.keys())
df_network

,source_id,target_id,weight,source_name,target_name
0737-01_1239-01,0737-01,1239-01,157,新橋駅前,浜離宮前
1239-01_2024-01,1239-01,2024-01,157,浜離宮前,築地五丁目
2024-01_2579-01,2024-01,2579-01,157,築地五丁目,国立がん研究センター前
2579-01_0946-02,2579-01,0946-02,154,国立がん研究センター前,築地六丁目
0946-02_0326-02,0946-02,0326-02,154,築地六丁目,勝どき駅前
...,...,...,...,...,...
2598-04_2596-02,2598-04,2596-02,22,水産仲卸棟,市場前駅前
2596-01_0469-02,2596-01,0469-02,50,市場前駅前,お台場海浜公園駅前
0302-02_2326-04,0302-02,2326-04,50,東京国際クルーズターミナル駅前,東京テレポート駅前
2326-04_0302-01,2326-04,0302-01,55,東京テレポート駅前,東京国際クルーズターミナル駅前


In [9]:
df_network.reset_index()[['source_name', 'target_name', 'weight']].sort_values(['weight'], ignore_index=True, ascending=False)

,source_name,target_name,weight
0,築地,築地三丁目,1706
1,築地三丁目,築地六丁目,1706
2,築地,銀座四丁目,1693
3,築地三丁目,築地,1693
4,築地六丁目,築地三丁目,1693
...,...,...,...
4263,青山いきいきプラザ入口,青山一丁目駅前,1
4264,葛西水再生センター前,東京臨海病院前,1
4265,新宿四丁目,角筈二丁目,1
4266,本所一丁目,蔵前駅前,1


In [10]:
df_network_graph = df_network.reset_index()[['source_name', 'target_name', 'weight']].sort_values(['weight'], ignore_index=True, ascending=False)
width = df_network_graph['weight'] / df_network_graph['weight'].max() * 23.0
width = pd.Series([int(i) + 1 for i in width])
df_network_graph['weight'] = width
df_network_graph['source'] = df_network_graph['source_name']
df_network_graph['target'] = df_network_graph['target_name']
df_network_graph = df_network_graph[['source', 'target', 'weight']]
df_network_graph

,source,target,weight
0,築地,築地三丁目,24
1,築地三丁目,築地六丁目,24
2,築地,銀座四丁目,23
3,築地三丁目,築地,23
4,築地六丁目,築地三丁目,23
...,...,...,...
4263,青山いきいきプラザ入口,青山一丁目駅前,1
4264,葛西水再生センター前,東京臨海病院前,1
4265,新宿四丁目,角筈二丁目,1
4266,本所一丁目,蔵前駅前,1


In [11]:
g = nx.from_pandas_edgelist(df_network_graph, edge_attr=True, create_using=nx.DiGraph())
# g.edges

In [12]:
net = Network(notebook=True, width='100%', height='90vh', bgcolor="#222222", font_color="white", filter_menu=True, cdn_resources='in_line')
net.from_nx(g)
net.save_graph(f'{workspace_dir}/index.html')
# net.show(f'{workspace_dir}/index.html')